# <center> Building a database for crime reports 

# Goal of the project 

The goal of project is to create a database named 'crimes_db' with a table - boston_crimes - with apprporiate datatypes for storing the data from the 'boston.csv' file. We will be creating the table inside a schema named crimes. We will also create the readonly and readwrite groups with appropriate privileges. Finally, we will also need to create one user for each of these groups.

# Create database and schema 

We will start by creating a database for storing our crime data as well as schema for containing the tables. We will make database named 'crime_db' ans schema in it named 'crimes'. 

In [66]:
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True 
cursor = conn.cursor()

# Create database named crime_db
cursor.execute("CREATE DATABASE crime_db OWNER dq;")
conn.close()

# Connect to the crime_db and create shcema named crimes
conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True 
cursor = conn.cursor()

cursor.execute("CREATE SCHEMA crimes;")

# Read dataset

Before we start creating tables, we need to gather some data about 'boston.csv'. 

In [67]:
import csv

with open('boston.csv') as file :
    reader = csv.reader(file) 
    rows = [row for row in reader]
    col_headers = rows[0]
    first_row = rows[1]

print("Headers : {}".format(col_headers))

Headers : ['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


# Check number and length of values in columns

We need to identify the proper datatypes for the columns. To help us with that, we will use a function get_col_set() that, given the name of CSV file and a column index (starting from 0, computes a Python set with all disticnt values contained in that column.

This function will be useful for two reasons:

1. Checking whether an enumerated datatype might be a good choice for representing a column
2. Computoing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns 

In [68]:
def get_col_set(csv_filename, col_index) : 
    col_set = set()
    with open(csv_filename) as file : 
        reader = csv.reader(file) 
        for row in reader :
            val = row[col_index]
            if val not in col_set : 
                col_set.add(val) 
    return col_set 

In [69]:
# Compute each of seven columns the number of differnt values it comes 

for num in range(len(col_headers)) : 
    num_val = len(get_col_set('boston.csv', num))
    print("The number of {} : {}".format(col_headers[num], num_val))

The number of incident_number : 298330
The number of offense_code : 220
The number of description : 240
The number of date : 1178
The number of day_of_the_week : 8
The number of lat : 18178
The number of long : 18178


There are some columns with a low number of distinct values offense_code, description, and day_of_the week. Because offense_code's type is number, we need to consider description and day_of_the_week to know the longest word. 

In [70]:
# Compute the maximum length of any value in the description column

description_values = get_col_set('boston.csv', 2)

max_length = 0 
for col_values in description_values :
    if len(col_values) > max_length : 
        max_length = len(col_values) 

print("The maximum length of any value in the description column is {}".format(max_length))

The maximum length of any value in the description column is 58


Becuase the maximum length of any values in the description column is 58, we need to optimize datatype with VARCHAR(80). 

# Create table : boston_crimes 

Now we will create a table name 'boston_crimes' inside the crimes schema of crime_db database. We will change type of column date_of_the_week to enumerated type. Each of column names and data type when we create table is such as below.

| Col name | Data type | 
|:---:|:---:|
|incident_number|INTEGER|
|offense_code|INTEGER|
|description|VARCHAR(80)|
|date|DATE|
|day_of_the_week|ENUM|
|lat|decimal|
|long|decimal| 

In [71]:
# Identify suitable column names for table
print(col_headers)

# Identify the kind of data contained in each column 
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [72]:
# Create ENUM data type 
cursor.execute("CREATE TYPE day AS ENUM('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")

# Create table boston_crimes 
cursor.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(80),
        date DATE,
        day_of_the_week day,
        lat decimal,
        long decimal
);""")

# Load data into boston_crimes 

In [73]:
with open('boston.csv') as file : 
    cursor.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file) 

In [74]:
# check the result 
cursor.execute("SELECT * FROM crimes.boston_crimes LIMIT 1;")
boston_crimes = cursor.fetchall()
print(boston_crimes)

[(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053'))]


# Revoke public privileges 

Our goal is to create the two user groups that we have learned about : 'readonly' and 'readwrite'. 

In [75]:
# Make sure that there are no privileges inherited from the pulbic group 
cursor.execute("REVOKE ALL ON SCHEMA public FROM public;")
cursor.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

# Creating readonly and readwrite group 

The readonly group is supposed to only have privileges to perform SELECT queries. In contrast, we want to readwrite group to be able to perform SELECT, INSERT, DELETE, and UPDATE queries.

In [76]:
# Creating readonly group and give previleges 
cursor.execute("CREATE GROUP readonly NOLOGIN;")
cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cursor.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cursor.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

# Creating readwrite group and give previleges 
cursor.execute("CREATE GROUP readwrite NOLOGIN;")
cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cursor.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cursor.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

# Create user

We are nearly done with setting up our crime database. The only thing that we need to do is create users. 

In [79]:
cursor.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cursor.execute("GRANT readonly TO data_analyst;")

cursor.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cursor.execute("GRANT readwrite TO data_scientist;")

# Test setting up the database 

We will use SQL queries to check whether the objects have been created and that users are groups have the right privileges. We will query the pg_roles table to inspect privileges related to the database and the information_schema.table_privileges table to inspect table privileges.

- pg_roles Columns 
    - rolname : Role name 
    - rolsuper : Role ahs super privileges 
    - rolcreatedb : Role can create databases
    - rolcanlogin : Role can log in
    
- information_shcema.table_privileges
    - grantee, privilege_type

In [80]:
# Close the old connection and re open it 
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cursor = conn.cursor()

# Check user and groups 
cursor.execute("""
    SELECT rolname, rolsuper, rolcreatedb, rolcanlogin FROM pg_roles 
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
users = cursor.fetchall()

for user in users : 
    print(user)
print("\n")

# Check privileges 
cursor.execute("""
    SELECT grantee, privilege_type FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite')
""")
privileges = cursor.fetchall()

for privilege in privileges : 
    print(privilege)

('readonly', False, False, False)
('readwrite', False, False, False)
('data_analyst', False, False, True)
('data_scientist', False, False, True)


('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
